<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/ISCC_NBS_Color_Matcher_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from math import sqrt

# Define a function to calculate Delta-E between two LAB colors
def delta_e(lab1, lab2):
    return sqrt((lab1[0] - lab2[0]) ** 2 + (lab1[1] - lab2[1]) ** 2 + (lab1[2] - lab2[2]) ** 2)

# Load ISCC-NBS dataset with LAB values
iscc_nbs_df = pd.read_csv('iscc_nbs_lab_colors.csv')

# Function to predict the closest ISCC-NBS color for an input LAB color
def find_closest_iscc_nbs_color(input_lab):
    closest_color = None
    min_delta_e = float('inf')

    # Iterate through the dataset to find the color with the smallest Delta-E
    for index, row in iscc_nbs_df.iterrows():
        lab_color = (row['L'], row['A'], row['B'])
        delta_e_value = delta_e(input_lab, lab_color)

        if delta_e_value < min_delta_e:
            min_delta_e = delta_e_value
            closest_color = row

    return closest_color, min_delta_e

# Example usage: Input LAB color
input_lab_color = [50, 0, 0]  # Example LAB input
closest_color, delta_e_value = find_closest_iscc_nbs_color(input_lab_color)

# Extracting details from the closest color
closest_color_name = closest_color['Color Name']
closest_lab_color = [closest_color['L'], closest_color['A'], closest_color['B']]

# Convert LAB values to RGB for plotting (using a simple scaling for visualization)
def lab_to_rgb(lab_color):
    # Normalize the LAB values to be between 0 and 1 (simplified for visualization)
    return [lab_color[1] / 128 + 0.5, lab_color[2] / 128 + 0.5, lab_color[0] / 100]

input_rgb = lab_to_rgb(input_lab_color)
closest_rgb = lab_to_rgb(closest_lab_color)

# 1. Create an interactive plot for the color comparison
color_fig = go.Figure()

# Add the input color swatch
color_fig.add_trace(go.Bar(
    x=['Input Color'],
    y=[1],
    marker=dict(color=f'rgb({input_rgb[0] * 255}, {input_rgb[1] * 255}, {input_rgb[2] * 255})'),
    name='Input Color',
    hoverinfo='text',
    text=f'LAB: {input_lab_color}'
))

# Add the closest match color swatch
color_fig.add_trace(go.Bar(
    x=['Closest Match'],
    y=[1],
    marker=dict(color=f'rgb({closest_rgb[0] * 255}, {closest_rgb[1] * 255}, {closest_rgb[2] * 255})'),
    name=f'Closest Match: {closest_color_name}',
    hoverinfo='text',
    text=f'LAB: {closest_lab_color}\nDelta-E: {delta_e_value:.2f}'
))

color_fig.update_layout(
    title=f'Input vs Closest Color (Delta-E: {delta_e_value:.2f})',
    xaxis_title='Colors',
    yaxis_title='Color Intensity',
    showlegend=False
)

# 2. Create an interactive LAB value comparison
lab_comparison_fig = go.Figure()

# Add the input LAB values
lab_comparison_fig.add_trace(go.Bar(
    x=['L', 'A', 'B'],
    y=input_lab_color,
    name='Input LAB',
    marker_color='blue',
    hoverinfo='text',
    text=f'Input LAB: {input_lab_color}'
))

# Add the closest LAB values
lab_comparison_fig.add_trace(go.Bar(
    x=['L', 'A', 'B'],
    y=closest_lab_color,
    name=f'Closest LAB: {closest_color_name}',
    marker_color='green',
    hoverinfo='text',
    text=f'Closest LAB: {closest_lab_color}'
))

lab_comparison_fig.update_layout(
    title='LAB Value Comparison',
    xaxis_title='LAB Components',
    yaxis_title='Values',
    barmode='group'
)

# Display the figures
color_fig.show()
lab_comparison_fig.show()

# 3. Print the details in a table format
print(f"\nInput LAB Color: {input_lab_color}")
print(f"Closest ISCC-NBS Color: {closest_color_name}")
print(f"Delta-E Value: {delta_e_value}")
print(f"LAB of Closest Color: {closest_lab_color}")
